# Kubernetes 101 Hands-on Workshop

This comprehensive workshop is designed to help you get started with Kubernetes, understand its basic constructs, and perform hands-on operations.

## Workshop Overview

1. **Setting up Kubernetes**
2. **Understanding Kubernetes Architecture**
3. **Zero to One - Quick Start**
4. **Basic Kubernetes Constructs**
5. **Configuration Management**
6. **Services and Networking**
7. **Advanced Topics**

**Prerequisites:**
- Basic understanding of containers and Docker
- Access to a Kubernetes cluster (minikube, Docker Desktop, or cloud provider)
- kubectl CLI tool installed

## 1. Setting up Kubernetes

Choose one of the following options for your Kubernetes cluster:

1. **Docker for Desktop** with Kubernetes enabled
2. **Minikube**
3. **Cloud Provider** (GKE, EKS, AKS)

### Installing Minikube

#### macOS
```bash
brew install minikube
```

#### Linux
```bash
curl -LO https://storage.googleapis.com/minikube/releases/latest/minikube-linux-amd64 \
  && sudo install minikube-linux-amd64 /usr/local/bin/minikube
```

#### Windows
Follow the [official installation guide](https://kubernetes.io/docs/tasks/tools/install-kubectl-windows/)

### Installing kubectl
Follow the [kubectl installation guide](https://kubernetes.io/docs/tasks/tools/install-kubectl)

In [ ]:
# Verify your Kubernetes installation
!kubectl version --client
!kubectl cluster-info

## 2. Kubernetes Architecture

Kubernetes follows a master-worker architecture:

### Master Components (Control Plane)
- **API Server**: Entry point for all REST commands
- **etcd**: Distributed key-value store
- **Controller Manager**: Runs controller processes
- **Scheduler**: Assigns pods to nodes

### Worker Node Components
- **kubelet**: Agent that communicates with master
- **kube-proxy**: Network proxy
- **Container Runtime**: Docker, containerd, etc.

### Kubernetes Capabilities
- **Service Discovery & Load Balancing**
- **Storage Orchestration**
- **Automated Rollouts & Rollbacks**
- **Self-healing**
- **Secret & Configuration Management**
- **Horizontal Scaling**

In [ ]:
# Explore your cluster
!kubectl get nodes
!kubectl get nodes -o wide

## 3. Zero to One - Quick Start

### Deploy Nginx in 60 seconds

In [ ]:
# Create a simple nginx deployment
!kubectl create deployment nginx --image=nginx

# Check the deployment
!kubectl get deployments
!kubectl get pods

In [ ]:
# Expose the deployment as a service
!kubectl expose deployment nginx --port=80 --type=LoadBalancer

# Check the service
!kubectl get services

In [ ]:
# Clean up
!kubectl delete service nginx
!kubectl delete deployment nginx

## 4. Basic Kubernetes Constructs

### 4.1 Pods - The Basic Building Block

- Smallest deployable unit in Kubernetes
- Group of containers that share network and storage
- Usually contains a single container
- Pods are ephemeral - they can die anytime!

In [ ]:
# Create a simple pod using kubectl run
!kubectl run my-pod --image=nginx --restart=Never

# Check the pod
!kubectl get pods
!kubectl describe pod my-pod

#### Pod YAML Definition

Let's create a pod using a YAML manifest:

In [ ]:
# Create a pod YAML file
pod_yaml = """
apiVersion: v1
kind: Pod
metadata:
  name: nginx-pod
  labels:
    app: nginx
spec:
  containers:
  - name: nginx
    image: nginx:1.20
    ports:
    - containerPort: 80
"""

# Write to file
with open('nginx-pod.yaml', 'w') as f:
    f.write(pod_yaml)

print("Pod YAML created!")

In [ ]:
# Apply the pod manifest
!kubectl apply -f nginx-pod.yaml

# Check the pod
!kubectl get pods -l app=nginx

In [ ]:
# Get detailed information about the pod
!kubectl describe pod nginx-pod

In [ ]:
# Access the pod logs
!kubectl logs nginx-pod

In [ ]:
# Execute commands inside the pod
!kubectl exec nginx-pod -- ls -la /usr/share/nginx/html

### 4.2 ReplicaSets

ReplicaSets ensure that a specified number of pod replicas are running at any given time.

In [ ]:
# Create a ReplicaSet YAML
replicaset_yaml = """
apiVersion: apps/v1
kind: ReplicaSet
metadata:
  name: nginx-replicaset
  labels:
    app: nginx
spec:
  replicas: 3
  selector:
    matchLabels:
      app: nginx
  template:
    metadata:
      labels:
        app: nginx
    spec:
      containers:
      - name: nginx
        image: nginx:1.20
        ports:
        - containerPort: 80
"""

with open('nginx-replicaset.yaml', 'w') as f:
    f.write(replicaset_yaml)

print("ReplicaSet YAML created!")

In [ ]:
# Apply the ReplicaSet
!kubectl apply -f nginx-replicaset.yaml

# Check ReplicaSets and Pods
!kubectl get replicasets
!kubectl get pods -l app=nginx

In [ ]:
# Test self-healing: delete a pod and watch it get recreated
!kubectl delete pod $(kubectl get pods -l app=nginx -o jsonpath='{.items[0].metadata.name}')

# Check pods again
!kubectl get pods -l app=nginx

### 4.3 Deployments

Deployments provide declarative updates for Pods and ReplicaSets. They manage rolling updates and rollbacks.

In [ ]:
# Create a Deployment YAML
deployment_yaml = """
apiVersion: apps/v1
kind: Deployment
metadata:
  name: nginx-deployment
  labels:
    app: nginx
spec:
  replicas: 3
  selector:
    matchLabels:
      app: nginx
  template:
    metadata:
      labels:
        app: nginx
    spec:
      containers:
      - name: nginx
        image: nginx:1.20
        ports:
        - containerPort: 80
"""

with open('nginx-deployment.yaml', 'w') as f:
    f.write(deployment_yaml)

print("Deployment YAML created!")

In [ ]:
# Clean up previous resources first
!kubectl delete -f nginx-replicaset.yaml --ignore-not-found
!kubectl delete pod nginx-pod --ignore-not-found
!kubectl delete pod my-pod --ignore-not-found

In [ ]:
# Apply the Deployment
!kubectl apply -f nginx-deployment.yaml

# Check Deployments, ReplicaSets, and Pods
!kubectl get deployments
!kubectl get replicasets
!kubectl get pods

In [ ]:
# Perform a rolling update
!kubectl set image deployment/nginx-deployment nginx=nginx:1.21

# Watch the rollout
!kubectl rollout status deployment/nginx-deployment

In [ ]:
# Check rollout history
!kubectl rollout history deployment/nginx-deployment

In [ ]:
# Rollback to previous version
!kubectl rollout undo deployment/nginx-deployment

# Check the rollout status
!kubectl rollout status deployment/nginx-deployment

### 4.4 Services

Services provide stable network endpoints for accessing pods. They enable service discovery and load balancing.

In [ ]:
# Create a Service YAML
service_yaml = """
apiVersion: v1
kind: Service
metadata:
  name: nginx-service
spec:
  selector:
    app: nginx
  ports:
    - protocol: TCP
      port: 80
      targetPort: 80
  type: ClusterIP
"""

with open('nginx-service.yaml', 'w') as f:
    f.write(service_yaml)

print("Service YAML created!")

In [ ]:
# Apply the Service
!kubectl apply -f nginx-service.yaml

# Check the service
!kubectl get services
!kubectl describe service nginx-service

In [ ]:
# Test the service from inside the cluster
!kubectl run test-pod --image=curlimages/curl --rm -it --restart=Never -- curl nginx-service

## 5. Configuration Management

### 5.1 ConfigMaps

ConfigMaps store configuration data as key-value pairs.

In [ ]:
# Create a ConfigMap from command line
!kubectl create configmap app-config --from-literal=database_url=postgresql://localhost:5432/myapp --from-literal=debug=true

# Check the ConfigMap
!kubectl get configmaps
!kubectl describe configmap app-config

In [ ]:
# Create a ConfigMap YAML
configmap_yaml = """
apiVersion: v1
kind: ConfigMap
metadata:
  name: nginx-config
data:
  nginx.conf: |
    events {
        worker_connections 1024;
    }
    http {
        server {
            listen 80;
            location / {
                return 200 'Hello from ConfigMap!';
                add_header Content-Type text/plain;
            }
        }
    }
"""

with open('nginx-configmap.yaml', 'w') as f:
    f.write(configmap_yaml)

!kubectl apply -f nginx-configmap.yaml

### 5.2 Secrets

Secrets store sensitive data such as passwords, OAuth tokens, and SSH keys.

In [ ]:
# Create a Secret
!kubectl create secret generic app-secret --from-literal=username=admin --from-literal=password=supersecret

# Check the Secret
!kubectl get secrets
!kubectl describe secret app-secret

## 6. Advanced Workload Types

### 6.1 DaemonSets

DaemonSets ensure that a copy of a Pod runs on all (or some) nodes in the cluster.

In [ ]:
# Create a DaemonSet YAML
daemonset_yaml = """
apiVersion: apps/v1
kind: DaemonSet
metadata:
  name: fluentd-daemonset
spec:
  selector:
    matchLabels:
      name: fluentd
  template:
    metadata:
      labels:
        name: fluentd
    spec:
      containers:
      - name: fluentd
        image: fluentd:v1.14-1
        resources:
          limits:
            memory: 200Mi
          requests:
            cpu: 100m
            memory: 200Mi
"""

with open('fluentd-daemonset.yaml', 'w') as f:
    f.write(daemonset_yaml)

print("DaemonSet YAML created!")

### 6.2 Jobs

Jobs run pods to completion for batch workloads.

In [ ]:
# Create a Job YAML
job_yaml = """
apiVersion: batch/v1
kind: Job
metadata:
  name: pi-calculation
spec:
  template:
    spec:
      containers:
      - name: pi
        image: perl:5.34.0
        command: ["perl", "-Mbignum=bpi", "-wle", "print bpi(2000)"]
      restartPolicy: Never
  backoffLimit: 4
"""

with open('pi-job.yaml', 'w') as f:
    f.write(job_yaml)

# Apply and run the job
!kubectl apply -f pi-job.yaml

# Check the job
!kubectl get jobs
!kubectl get pods

In [ ]:
# Check job logs
import time
time.sleep(10)  # Wait for job to complete
!kubectl logs -l job-name=pi-calculation

## 7. Useful kubectl Commands

Here are some essential kubectl commands for daily operations:

In [ ]:
# Get all resources in current namespace
!kubectl get all

# Get resources across all namespaces
!kubectl get pods --all-namespaces

# Get resource with more details
!kubectl get pods -o wide

# Get resource in YAML format
!kubectl get deployment nginx-deployment -o yaml

In [ ]:
# Resource usage
!kubectl top nodes 2>/dev/null || echo "Metrics server not available"
!kubectl top pods 2>/dev/null || echo "Metrics server not available"

## 8. Cleanup

Let's clean up all the resources we created during this workshop:

In [ ]:
# Delete all created resources
!kubectl delete -f nginx-deployment.yaml --ignore-not-found
!kubectl delete -f nginx-service.yaml --ignore-not-found
!kubectl delete -f nginx-configmap.yaml --ignore-not-found
!kubectl delete -f pi-job.yaml --ignore-not-found
!kubectl delete configmap app-config --ignore-not-found
!kubectl delete secret app-secret --ignore-not-found

# Clean up YAML files
import os
yaml_files = ['nginx-pod.yaml', 'nginx-replicaset.yaml', 'nginx-deployment.yaml', 
              'nginx-service.yaml', 'nginx-configmap.yaml', 'fluentd-daemonset.yaml', 'pi-job.yaml']

for file in yaml_files:
    if os.path.exists(file):
        os.remove(file)
        print(f"Removed {file}")

print("\nCleanup completed!")

## 9. Next Steps

Congratulations! You've completed the Kubernetes 101 workshop. Here are some next steps to continue your Kubernetes journey:

### Additional Topics to Explore:
- **Namespaces**: Organize resources and provide isolation
- **Persistent Volumes**: Manage storage for stateful applications
- **StatefulSets**: Deploy stateful applications
- **Ingress Controllers**: Manage external access to services
- **Network Policies**: Control network traffic between pods
- **RBAC**: Role-based access control
- **Helm**: Package manager for Kubernetes
- **Monitoring & Logging**: Prometheus, Grafana, ELK stack
- **CI/CD**: GitOps with ArgoCD or Flux

### Recommended Resources:
- [Kubernetes Official Documentation](https://kubernetes.io/docs/)
- [Kubernetes the Hard Way](https://github.com/kelseyhightower/kubernetes-the-hard-way)
- [CNCF Landscape](https://landscape.cncf.io/)
- [Kubernetes Slack Community](https://kubernetes.slack.com/)

### Practice Environments:
- [Katacoda Kubernetes Scenarios](https://www.katacoda.com/courses/kubernetes)
- [Play with Kubernetes](https://labs.play-with-k8s.com/)
- [Google Cloud Shell](https://shell.cloud.google.com/)